<a href="https://colab.research.google.com/github/shivg7706/BrainTumorSegmentation/blob/master/BrainTumorSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp ./drive/My\ Drive/seg_data.zip ./

In [0]:
!unzip seg_data.zip
!rm -rf seg_data.zip

## Dataset


In [0]:
import requests
from tqdm import tqdm_notebook as tqdm
import os
import argparse



def download_file(url, path):

    if check_if_file_exits(path):
        print(f'Already existing file {path}')
        return

    # Deleting the partial downloaded file.
    if os.path.isfile(path):
        print(f'Deleted existing partial file {path}')
        os.remove(path)

    response = requests.get(url, stream=True)
    handle = open(path, "wb")
    with open(path, "wb") as handle:
        chunk_size = 1024
        total_size = round(int(response.headers['Content-Length']), 3)
        pbar = tqdm(unit="B", total=total_size)
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:  # filter out keep-alive new chunks
                handle.write(chunk)
                pbar.update(len(chunk))


    extension = path[-3:]
    os.rename(path, path[:-4]+'_done.'+extension)


def make_folder(target_folder):

    if not (os.path.isdir(target_folder)):
        print(f'Creating {target_folder} folder')
        os.mkdir(target_folder)


def main():

    dataset_urls = ['https://ndownloader.figshare.com/files/3381290',
                    'https://ndownloader.figshare.com/files/3381296',
                    'https://ndownloader.figshare.com/files/3381293',
                    'https://ndownloader.figshare.com/files/3381302']

    dataset_readme = 'https://ndownloader.figshare.com/files/7953679'

    target_folder = 'dataset'
    dataset_part = 1
    dataset_file_name = f'brain_tumor_dataset_part_'

    make_folder(target_folder)

    print('Downloading dataset README.txt\n')
    download_file(dataset_readme, os.path.join(target_folder, 'README.TXT'))

    print('Starting download process\n')
    for url in dataset_urls:
        try:
            path = os.path.join(
                target_folder, f'{dataset_file_name}{dataset_part}.zip')
            print(f'\t\tDownloading :  {path}')
            download_file(url, path)
            dataset_part += 1



main()

## Converting Data from `.mat` files into images

In [0]:
import h5py
import numpy as np
import tensorflow as tftype('float32')

In [0]:
mapping = {
    1: 'meningioma',
    2: 'glioma',
    3: 'pituitary'
}

In [0]:
def save_it(data):
    
    mpimg.imsave('./tumorImage/{}_tumor_{}.png'.format(data['count'], data['label']), data['image'], cmap='gray', format='png')
    mpimg.imsave('./maskImage/{}_mask_{}.png'.format(data['count'], data['label']), data['mask'], cmap='gray', format='png')
    

In [0]:
import matplotlib.image as mpimg

for file in tqdm(os.listdir('./dataset')):
    f = h5py.File('./dataset/{}'.format(file))
    data = dict()
    data['count'] = file[:-4]
    data['image'] = np.array(f.get('cjdata/image'))
    data['mask'] = np.array(f.get('cjdata/tumorMask'))
    data['label'] = mapping[np.array(f.get('cjdata/label'))[0, 0]]

    save_it(data)
        
        

In [0]:
import os
for i in range(3001, 3065):
    os.rename('./tumor/{}.png'.format(i), '../test_tumor/{}.png'.format(i))

In [0]:
!mv ../test/ ../test_mask

In [0]:
for file in os.listdir('segmentation_dataset/maskImage/'):
    if not os.path.exists('segmentation_dataset/maskImage/mask'):
        os.mkdir('segmentation_dataset/maskImage/mask')
    os.rename('segmentation_dataset/maskImage/{}'.format(file), 'segmentation_dataset/maskImage/mask/{}'.format(file))

In [0]:
import cv2
import numpy as np

seed = 1
np.random.seed(seed)

def my_generator(batch_size=4):
    
#     while True:
        tumors = []
        masks = []
        choose = np.random.randint(1, 3000, size=batch_size)
#         print(choose)
        for i in choose:
            
            tumor = cv2.imread('segmentation_dataset/tumorImage/tumor/{}.png'.format(i), 0) / 255
#             tumor = cv2.resize(tumor, (256, 256)) / 255
            plt.imshow(tumor, cmap='gray')
            plt.draw()
            plt.pause(0.2)
            tumor = tumor[:, :, np.newaxis]
            tumors.append(tumor)
            
            mask = cv2.imread('segmentation_dataset/maskImage/mask/{}.png'.format(i), 0) / 255
#             mask = cv2.resize(tumor, (256, 256)) / 255
            plt.imshow(mask, cmap='gray')
            plt.draw()
            plt.pause(0.2)
            mask = mask[:, :, np.newaxis]
            masks.append(mask)
            
        
        tumors = np.asarray(tumors)
#         print(tumors.shape)
        masks = np.asarray(masks)
#         if tumors.ndim == 4:
#             yield (tumors, masks)
#             
            

In [0]:
import os
import cv2

for img in os.listdir('./segmentation_dataset/maskImage/mask/'):
    
    imag = cv2.imread('./segmentation_dataset/maskImage/mask/'  + img, 0)
    print(imag.shape)
    os.system('clear')
    if imag.shape == (256, 256):
        os.remove('./segmentation_dataset/maskImage/mask/{}'.format(img))
        os.remove('./segmentation_dataset/tumorImage/tumor/{}'.format(img))
    

In [0]:
img1 = cv2.imread('segmentation_dataset/tumorImage/tumor/300.png', 0)/255
img1 = img1[:, :, np.newaxis]

In [0]:
def combine(img_gen, mask_gen):
    while True:
        yield(img_gen.next(), mask_gen.next())

In [0]:
import cv2
import os
import random

def data_gen(img_folder, mask_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    print(n)
    random.shuffle(n)
  
    while (True):
        img = np.zeros((batch_size, 512, 512, 3)).astype('float')
        mask = np.zeros((batch_size, 512, 512, 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

            train_img = cv2.imread(img_folder+'/'+n[i])/255.
            train_img =  cv2.resize(train_img, (512, 512))# Read an image from folder and resize

            img[i-c] = train_img #add to array - img[0], img[1], and so on.


            train_mask = cv2.imread(mask_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
            train_mask = cv2.resize(train_mask, (512, 512))
            train_mask = train_mask.reshape(512, 512, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

            mask[i-c] = train_mask

        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            random.shuffle(n)
                  # print "randomizing again"
        yield img, mask




train_frame_path = './segmentation_dataset/tumorImage/tumor/'
train_mask_path = './segmentation_dataset/maskImage/mask/'

# val_frame_path = '/path/to/validation_frames'
# val_mask_path = '/path/to/validation_frames'

# Train the model
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4)
# val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4)

## Training part

In [19]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

data_gen_args = dict(rescale=1./255)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1

img_gen = image_datagen.flow_from_directory('segmentation_dataset/tumorImage/', batch_size=1, target_size=(512, 512),
                                       class_mode=None, seed=seed, color_mode='grayscale', shuffle=True)
mask_gen = mask_datagen.flow_from_directory('segmentation_dataset/maskImage/', batch_size=1, target_size=(512, 512),
                                       class_mode=None, seed=seed, color_mode='grayscale', shuffle=True)

train_gen = combine(img_gen, mask_gen)

Found 3064 images belonging to 1 classes.
Found 3064 images belonging to 1 classes.


In [0]:
from tensorflow.python.keras.layers.convolutional import Conv2D, ZeroPadding2D, Conv2DTranspose
from tensorflow.python.keras.layers.pooling import MaxPool2D, AvgPool2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers import Dense, Flatten, Input, BatchNormalization, Dropout, Activation, Concatenate, concatenate, Add, UpSampling2D
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.backend as K
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.utils import plot_model

In [6]:
from keras.layers import LeakyReLU
from tensorflow.python.keras.optimizers import Adam

def get_conv_block(input, filter):
    
    x = input
    
    for i in range(2):
        x = Conv2D(filter, 3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
    return x

Using TensorFlow backend.


In [0]:
def unet():
    
    f = [16, 32, 64, 128, 256]
    
    inputs = Input((512, 512, 1), name='input')
    
    # Encoder
    block1 = get_conv_block(inputs, f[0])
    x = MaxPool2D(2)(block1)
    
    block2 = get_conv_block(x, f[1])
    x = MaxPool2D(2)(block2)
    
    block3 = get_conv_block(x, f[2])
    x = MaxPool2D(2)(block3)
    
    block4 = get_conv_block(x, f[3])
    
    # Middle part
    
    x = MaxPool2D(2)(block4)
    x = get_conv_block(x, f[4])
    
    # Decoder
    
    x = Conv2DTranspose(f[3], kernel_size=2, strides=2)(x)
    x = Concatenate()([block4, x])
    x = get_conv_block(x, f[3])
    
    x = Conv2DTranspose(f[2], kernel_size=2, strides=2)(x)
    x = Concatenate()([block3, x])
    x = get_conv_block(x, f[2])
    
    x = Conv2DTranspose(f[1], kernel_size=2, strides=2)(x)
    x = Concatenate()([block2, x])
    x = get_conv_block(x, f[1])
    
    x = Conv2DTranspose(f[0], kernel_size=2, strides=2)(x)
    x = Concatenate()([block1, x])
    x = get_conv_block(x, f[0])
    
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(x)
    opt = Adam(lr=1e-4)
    model = Model(inputs, outputs)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[dice_coef])
    
    return model

In [0]:
def iou_loss_core(y_true, y_pred, smooth=0.005):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
    
    return coef

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [0]:
from tensorflow.python.ops import clip_ops
def _constant_to_tensor(x, dtype):

    return constant_op.constant(x, dtype=dtype)


def loss_tensor(y_true, y_pred):
    
    epsilon_ = _constant_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1.0-epsilon_)
    out = -(y_true * K.log(y_pred) + (1.0 - y_true) * K.log(1.0 - y_pred))
    return K.mean(out, axis=-1)
     

In [24]:
model = ResUNet()
model.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d_60 (Conv2D)              (None, 512, 512, 16) 160         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 512, 512, 16) 64          conv2d_60[0][0]                  
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 512, 512, 16) 0           batch_normalization_56[0][0]     
____________________________________________________________________________________________

## ResUNet

In [0]:
def bn_act(x, act=True):
    x = BatchNormalization()(x)
    if act == True:
        x = Activation("relu")(x)
    return x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    
    shortcut = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)
    
    shortcut = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = UpSampling2D((2, 2))(x)
    c = Concatenate()([u, xskip])
    return c

In [0]:
def ResUNet():
    f = [16, 32, 64, 128, 256]
    inputs = Input((512, 512, 1))
    
    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])
    
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
    model = Model(inputs, outputs)
    plot_model(model, to_file='ResUNet.png')
    
    return model

In [34]:
# history = model.fit(X, y, epochs=20, shuffle=True, validation_split=0.1, batch_size=8)
history = model.fit_generator(train_gen, shuffle=True, steps_per_epoch=750, epochs=10)

Epoch 1/10
750/750 [==============================] - 109s 145ms/step - loss: 0.4229 - dice_coef: 0.5771
Epoch 2/10
750/750 [==============================] - 111s 148ms/step - loss: 0.4033 - dice_coef: 0.5967
Epoch 3/10
750/750 [==============================] - 111s 148ms/step - loss: 0.3752 - dice_coef: 0.6248
Epoch 4/10
750/750 [==============================] - 111s 148ms/step - loss: 0.3893 - dice_coef: 0.6107
Epoch 5/10
750/750 [==============================] - 110s 147ms/step - loss: 0.3686 - dice_coef: 0.6314
Epoch 6/10
750/750 [==============================] - 111s 148ms/step - loss: 0.3662 - dice_coef: 0.6338
Epoch 7/10
750/750 [==============================] - 111s 147ms/step - loss: 0.3380 - dice_coef: 0.6620
Epoch 8/10
750/750 [==============================] - 110s 147ms/step - loss: 0.3581 - dice_coef: 0.6419
Epoch 9/10
750/750 [==============================] - 111s 148ms/step - loss: 0.3339 - dice_coef: 0.6661
Epoch 10/10
750/750 [==============================] - 

In [0]:
model.save('./drive/My Drive/BTS_final.h5')

In [0]:
import cv2
img = cv2.imread('segmentation_dataset/tumorImage/tumor/151.png', 0) / 255.
import matplotlib.pyplot as plt
plt.imshow(img, cmap='gray')

In [0]:
import numpy as np
# img = cv2.resize(img, (256, 256))
img = img.reshape(-1, 512, 512, 1)
y_pred = model.predict(img)

y = np.squeeze(y_pred)
plt.imshow(y, cmap='gray')

In [43]:
!nvidia-smi

Wed Jul 24 10:06:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    28W /  70W |  14737MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  